In [0]:
!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6

Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
     |████████████████████████████████| 1.5MB 2.8MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 58 kB 1.7 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-py3-none-any.whl size=72859 sha256=9a7178b681ea16de9a3c1dd526857fe7d9abddcf518d36abeaa94cef0fa09a43
  Stored in directory: /root/.cache/pip/wheels/01/3e/ff/77407ebac3ef71a79b9166a8382aecf88415a0bcbe3c095a01
Successfully built kaggle


In [36]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
!kaggle -v

Kaggle API 1.5.6


In [0]:
!kaggle competitions download -c 2020-ai-exam-cabbage

!unzip 2020-ai-exam-cabbage.zip

  0% 0.00/29.7k [00:00<?, ?B/s]
100% 29.7k/29.7k [00:00<00:00, 22.5MB/s]
Archive:  2020-ai-exam-cabbage.zip
  inflating: submit_sample.csv       
  inflating: test_cabbage_price.csv  
  inflating: train_cabbage_price.csv  


In [0]:
import numpy as np
import torch 
import torch.optim as optim

xy=np.loadtxt('train_cabbage_price.csv', delimiter=",", dtype=np.float32, skiprows=1, usecols=range(1,6))

x_train=torch.from_numpy(xy[:,0:-1])
y_train=torch.from_numpy(xy[:,[-1]])

print(x_train[:5])
print(y_train[:5])

tensor([[ -4.9000, -11.0000,   0.9000,   0.0000],
        [ -3.1000,  -5.5000,   5.5000,   0.8000],
        [ -2.9000,  -6.9000,   1.4000,   0.0000],
        [ -1.8000,  -5.1000,   2.2000,   5.9000],
        [ -5.2000,  -8.7000,  -1.8000,   0.7000]])
tensor([[2123.],
        [2123.],
        [2123.],
        [2020.],
        [2060.]])


# 모델 학습

In [0]:
W=torch.zeros((4,1),requires_grad=True)
b=torch.zeros(1,requires_grad=True)

optimizer = optim.SGD([W,b], lr=1e-5)

nb_epochs =100000
for epoch in range(nb_epochs+1):

  hypothesis = x_train.matmul(W)+b
  cost=torch.mean((hypothesis-y_train)**2)

  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch % 10000==0:
    print('Epoch {:4d}/{}  hypothesis: {} Cost:{:.6f}'.format(
        epoch, nb_epochs, hypothesis.squeeze().detach(), cost.item()
    ))

Epoch    0/100000  hypothesis: tensor([0., 0., 0.,  ..., 0., 0., 0.]) Cost:12610017.000000
Epoch 10000/100000  hypothesis: tensor([2549.9980, 2603.2512, 1834.0828,  ..., 2503.3813, 2199.7976,
        1948.1005]) Cost:2271455.000000
Epoch 20000/100000  hypothesis: tensor([2580.5649, 2658.6826, 1866.2841,  ..., 2525.0493, 2220.6553,
        1970.2579]) Cost:2260082.500000
Epoch 30000/100000  hypothesis: tensor([2582.0178, 2689.6987, 1878.6428,  ..., 2529.7480, 2226.8857,
        1978.7141]) Cost:2249238.250000
Epoch 40000/100000  hypothesis: tensor([2583.1519, 2719.5999, 1890.6082,  ..., 2534.2126, 2232.9170,
        1986.9585]) Cost:2238702.500000
Epoch 50000/100000  hypothesis: tensor([2584.1924, 2748.4783, 1902.3284,  ..., 2538.5723, 2238.8672,
        1995.1045]) Cost:2228467.750000
Epoch 60000/100000  hypothesis: tensor([2585.5356, 2776.8015, 1914.0858,  ..., 2543.0754, 2244.9436,
        2003.3423]) Cost:2218519.500000
Epoch 70000/100000  hypothesis: tensor([2586.7417, 2804.2952, 1

In [0]:
xy_test=np.loadtxt('test_cabbage_price.csv', delimiter=",", dtype=np.float32, skiprows=1, usecols=range(1,5))

test_x = torch.from_numpy(xy_test)
print(test_x)

tensor([[13.5000,  8.0000, 19.2000,  2.6000],
        [ 1.1000, -3.0000,  9.5000,  0.8000],
        [12.0000,  5.1000, 19.6000,  0.0000],
        [16.6000, 11.9000, 22.5000,  0.1000],
        [22.4000, 17.8000, 27.4000,  0.5000],
        [18.8000, 13.9000, 24.4000,  0.0000]])


In [0]:
prediction = test_x.matmul(W)+b

print(test_x)
print(prediction)

tensor([[13.5000,  8.0000, 19.2000,  2.6000],
        [ 1.1000, -3.0000,  9.5000,  0.8000],
        [12.0000,  5.1000, 19.6000,  0.0000],
        [16.6000, 11.9000, 22.5000,  0.1000],
        [22.4000, 17.8000, 27.4000,  0.5000],
        [18.8000, 13.9000, 24.4000,  0.0000]])
tensor([[3327.4626],
        [3257.0093],
        [3883.8081],
        [3295.0090],
        [3294.5144],
        [3332.1853]], grad_fn=<AddBackward0>)


In [0]:
import pandas as pd

submit=pd.read_csv("submit_sample.csv")
submit

,Id,Expected
0,0,1000
1,1,1000
2,2,1000
3,3,1000
4,4,1000
5,5,1000


In [0]:
for i in range(len(prediction)):
  submit['Expected'][i]=prediction[i].item()

submit

,Id,Expected
0,0,3327
1,1,3257
2,2,3883
3,3,3295
4,4,3294
5,5,3332


In [0]:
submit.to_csv('submit.csv', mode='w', index=False)


In [0]:
! kaggle competitions submit  -c 2020-ai-exam-cabbage -f submit.csv -m '3 submit'

100% 54.0/54.0 [00:00<00:00, 160B/s]
Successfully submitted to 2020.AI.중간고사.문제1